<a href="https://colab.research.google.com/github/hws2002/Master-Python-with-Just-One-Book-ADP/blob/main/Chapter7/Chapter7_4_chai_squared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 적합성 검정
titanic데이터에서 sex변수에 대한 분할표를 생성하고 아래의 가설에 대한 적합도 검정을 수행해보자.  
귀무가설 : 타이나틱호의 생존자 중 남자의 비율이 50%, 여자의 비율이 50%이다.  
대립가설 : 타이타닉호의 생존자 중 남자의 비율이 50%, 여자의 비율이 50%라고 할 수 없다.  

In [2]:
#데이터 불러오기
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/ADPclass/ADP_book_ver01/refs/heads/main/data/titanic.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   adult_male   891 non-null    bool   
 10  embark_town  889 non-null    object 
dtypes: bool(1), float64(2), int64(4), object(4)
memory usage: 70.6+ KB


In [3]:
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,adult_male,embark_town
0,0,3,male,22.0,1,0,7.2500,S,Third,True,Southampton
1,1,1,female,38.0,1,0,71.2833,C,First,False,Cherbourg
2,1,3,female,26.0,0,0,7.9250,S,Third,False,Southampton
3,1,1,female,35.0,1,0,53.1000,S,First,False,Southampton
4,0,3,male,35.0,0,0,8.0500,S,Third,True,Southampton
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,True,Southampton
887,1,1,female,19.0,0,0,30.0000,S,First,False,Southampton
888,0,3,female,NaN,1,2,23.4500,S,Third,False,Southampton
889,1,1,male,26.0,0,0,30.0000,C,First,True,Cherbourg


In [34]:
# 크로스탭 데이터 생성하기
## male데이터, female데이터 나누기
survived = df[df['survived'] == 0]
male = df.loc[df['sex'] == 'male', ['survived']]
female = df.loc[df['sex'] == 'female', ['survived']]

## 크로스탭 테이블 생성
survived_sex = pd.DataFrame({'male' : [ len(male[male['survived'] == 1]), len(male[male['survived'] == 0])],
                             'female' : [ len(female[female['survived'] == 1]), len(female[female['survived'] == 0])]})
survived_sex.index = ['survived', 'unsurvived']
survived_sex

,male,female
survived,109,233
unsurvived,468,81


In [45]:
# 적합도 검정을 수행해보자
# 귀무가설이 "타이타닉호의 생존자 중 남자의 비율과 여자의 비율이 50%, 50%로 같다"
# 이므로 타이타닉호에서 생존한 남자의 빈도수와, 여자의 빈도수를 f_obs로,
# 각 카테고리의 기대빈도인 [생존자수/2, 생존자수/2]를 f_exp로 설정해야한다
from scipy.stats import chisquare

total_survived = survived_sex.loc['survived'].sum(axis = 0)
chi = chisquare(survived_sex.loc['survived'], f_exp = [total_survived//2, total_survived//2])
print('<적합도 검정> \n', chi)

<적합도 검정> 
 Power_divergenceResult(statistic=44.95906432748538, pvalue=2.0119672574477235e-11)


카이제곱 통계량은 약 44.96, p-value값은 유의수준 0.5보다 낮으므로, 귀무가설을 기각한다.  
즉, 타이타닉호의 생존자 중 남자와 여자의 비율이 같다고 볼 수 없다.

# 2. 독립성 검정
titanic 데이터에서 좌석등급(class)과 생존 여부(survived)가 서로 독립인지 검정해보자.  
귀무가설 : class 변수와 survived 변수는 독립이다.  
대립가설 : class 변수와 survived 변수는 독립이 아니다.  

In [46]:
df = pd.read_csv('https://raw.githubusercontent.com/ADPclass/ADP_book_ver01/main/data/titanic.csv')
table = pd.crosstab(df['class'], df['survived'])
table

survived,0,1
class,,
First,80,136
Second,97,87
Third,372,119


In [48]:
# 독립성 검정을 하기 위해 크로스탭을 사용하여 교차표를 생성한다.
from scipy.stats import chi2_contingency
chi, p, df, expect = chi2_contingency(table)

print("Statistics : ",chi)
print("p-value : ",p)
print('df : ',df)
print("expect : \n", expect)

Statistics :  102.88898875696056
p-value :  4.549251711298793e-23
df :  2
expect : 
 [[133.09090909  82.90909091]
 [113.37373737  70.62626263]
 [302.53535354 188.46464646]]


독립성 검정 결과 p-value가 유의수준 0.05보다 작으므로 귀무가설을 기각한다.  
즉, 좌석 등급과 생존은 유의수준 0,05하에서 독립이 아니라고 할 수 있다.